In [1]:
import pandas as pd
import sqlite3
import json, csv
import geopandas
import re
import numpy as np
import time

In [4]:
address_points = pd.read_csv('cook_locations.csv')
address_points.head()

/tmp/ipykernel_441706/3716568878.py:1: DtypeWarning: Columns (1,5,7,11,17,25,26,27,30,31,32,36,38,40,46,47,48,52,54) have mixed types. Specify dtype option on import or set low_memory=False.
  address_points = pd.read_csv('cook_locations.csv')


,OBJECTID,AddNum_Pre,Add_Number,AddNum_Suf,ADDRNOCOM,St_PreMod,St_PreDir,St_PreTyp,St_Name,St_PosTyp,...,Seat,Inc_Muni,Uninc_Comm,MSAGComm,DiscrpAgID,Site_NGUID,ESN,TWP_NAME,LSt_PreDir,LSt_Type
0,1,NaN,1763.0,NaN,1763,NaN,WEST,NaN,WELLINGTON,AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,1622.0,NaN,1622,NaN,WEST,NaN,CULLERTON,STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,917.0,NaN,917,NaN,WEST,NaN,ARMITAGE,AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,20018.0,NaN,20018,NaN,NaN,NaN,MOHAWK,TRAIL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,20024.0,NaN,20024,NaN,NaN,NaN,MOHAWK,TRAIL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
address_points.columns

Index(['OBJECTID', 'AddNum_Pre', 'Add_Number', 'AddNum_Suf', 'ADDRNOCOM',
       'St_PreMod', 'St_PreDir', 'St_PreTyp', 'St_Name', 'St_PosTyp',
       'St_PosDir', 'St_PosMod', 'STNAMECOM', 'LandmkName', 'PLACENAME',
       'Post_Comm', 'State', 'Post_Code', 'Post_Code4', 'ADDRDELIV',
       'CMPADDABRV', 'Long', 'Lat', 'USGRIDCORD', 'Elev', 'PIN', 'Effective',
       'Expire', 'EDITTIME', 'geocode_muni', 'Document_Name', 'created_user',
       'created_date', 'last_edited_user', 'last_edited_date', 'Source',
       'Source_Comment', 'Structure', 'OrthoYear', 'Placement', 'County',
       'Building', 'Floor', 'Unit', 'Room', 'Seat', 'Inc_Muni', 'Uninc_Comm',
       'MSAGComm', 'DiscrpAgID', 'Site_NGUID', 'ESN', 'TWP_NAME', 'LSt_PreDir',
       'LSt_Type'],
      dtype='object')

In [6]:
cook_locations = address_points[['Add_Number', 'ADDRDELIV','CMPADDABRV','Lat', 'Long','PLACENAME','Post_Code']]
cook_locations.head()

,Add_Number,ADDRDELIV,CMPADDABRV,Lat,Long,PLACENAME,Post_Code
0,1763.0,1763 WEST WELLINGTON AVENUE,1763 W WELLINGTON AVE,41.935796,-87.673357,Chicago,60657
1,1622.0,1622 WEST CULLERTON STREET,1622 W CULLERTON ST,41.855258,-87.667139,Chicago,60608
2,917.0,917 WEST ARMITAGE AVENUE,917 W ARMITAGE AVE,41.917926,-87.651651,Chicago,60614
3,20018.0,20018 MOHAWK TRAIL,20018 MOHAWK TRL,41.525564,-87.695313,Olympia Fields,60461
4,20024.0,20024 MOHAWK TRAIL,20024 MOHAWK TRL,41.525317,-87.695269,Olympia Fields,60461


In [7]:
cook_locations.dtypes

Add_Number    float64
ADDRDELIV      object
CMPADDABRV     object
Lat           float64
Long          float64
PLACENAME      object
Post_Code      object
dtype: object

In [8]:
cook_locations.shape

(1347092, 7)

In [9]:
#get rid of nan values

In [10]:
cook_locations_clean = cook_locations[~cook_locations['Post_Code'].isnull()]
cook_locations_clean

,Add_Number,ADDRDELIV,CMPADDABRV,Lat,Long,PLACENAME,Post_Code
0,1763.0,1763 WEST WELLINGTON AVENUE,1763 W WELLINGTON AVE,41.935796,-87.673357,Chicago,60657
1,1622.0,1622 WEST CULLERTON STREET,1622 W CULLERTON ST,41.855258,-87.667139,Chicago,60608
2,917.0,917 WEST ARMITAGE AVENUE,917 W ARMITAGE AVE,41.917926,-87.651651,Chicago,60614
3,20018.0,20018 MOHAWK TRAIL,20018 MOHAWK TRL,41.525564,-87.695313,Olympia Fields,60461
4,20024.0,20024 MOHAWK TRAIL,20024 MOHAWK TRL,41.525317,-87.695269,Olympia Fields,60461
...,...,...,...,...,...,...,...
1347087,222.0,222 MERCHANDISE MART PLAZA,222 MERCHANDISE MART PLZ,NaN,NaN,CHICAGO,60654
1347088,1410.0,1410 SOUTH MUSEUM CAMPUS DRIVE,1410 S MUSEUM CAMPUS DR,NaN,NaN,CHICAGO,60605
1347089,NaN,6660 WEST 66TH STREET,6660 W 66TH ST,NaN,NaN,BEDFORD PARK,60638
1347090,9590.0,9590 WEST 55TH STREET,9590 W 55TH ST,NaN,NaN,COUNTRYSIDE,60525


In [11]:
#get rid of empty strings

In [14]:
cook_locations_clean[cook_locations_clean['Post_Code'] == ' ']

,Add_Number,ADDRDELIV,CMPADDABRV,Lat,Long,PLACENAME,Post_Code
1339273,400.0,NaN,400 N DEE RD,42.014499,-87.855737,,
1339292,4399.0,4399 PROSPECT AVE,4399 PROSPECT AVE,41.808459,-87.906042,,
1339294,1214.0,NaN,1214 HILLGROVE AVE,41.808673,-87.905517,,
1339295,1200.0,NaN,1200 HILLGROVE AVE,41.808770,-87.904980,,
1339296,4396.0,NaN,4396A CENTRAL AVE,41.808771,-87.904581,,
1339347,7936.0,NaN,7936 OGDEN AVE,41.819931,-87.820065,,
1339350,3919.0,NaN,3919 JOLIET AVE,41.820066,-87.820799,,
1339351,3921.0,NaN,3921 JOLIET AVE,41.820005,-87.820778,,
1339799,7504.0,NaN,7504 S CORK AVE,41.754061,-87.839178,,
1339800,7500.0,NaN,7500 S CORK AVE,41.754101,-87.839137,,


In [15]:
cook_locations_clean = cook_locations_clean[cook_locations_clean['Post_Code'] != ' ']
cook_locations_clean

,Add_Number,ADDRDELIV,CMPADDABRV,Lat,Long,PLACENAME,Post_Code
0,1763.0,1763 WEST WELLINGTON AVENUE,1763 W WELLINGTON AVE,41.935796,-87.673357,Chicago,60657
1,1622.0,1622 WEST CULLERTON STREET,1622 W CULLERTON ST,41.855258,-87.667139,Chicago,60608
2,917.0,917 WEST ARMITAGE AVENUE,917 W ARMITAGE AVE,41.917926,-87.651651,Chicago,60614
3,20018.0,20018 MOHAWK TRAIL,20018 MOHAWK TRL,41.525564,-87.695313,Olympia Fields,60461
4,20024.0,20024 MOHAWK TRAIL,20024 MOHAWK TRL,41.525317,-87.695269,Olympia Fields,60461
...,...,...,...,...,...,...,...
1347087,222.0,222 MERCHANDISE MART PLAZA,222 MERCHANDISE MART PLZ,NaN,NaN,CHICAGO,60654
1347088,1410.0,1410 SOUTH MUSEUM CAMPUS DRIVE,1410 S MUSEUM CAMPUS DR,NaN,NaN,CHICAGO,60605
1347089,NaN,6660 WEST 66TH STREET,6660 W 66TH ST,NaN,NaN,BEDFORD PARK,60638
1347090,9590.0,9590 WEST 55TH STREET,9590 W 55TH ST,NaN,NaN,COUNTRYSIDE,60525


In [16]:
cook_locations_clean.dtypes

Add_Number    float64
ADDRDELIV      object
CMPADDABRV     object
Lat           float64
Long          float64
PLACENAME      object
Post_Code      object
dtype: object

In [18]:
cook_locations_clean['Post_Code'] = cook_locations_clean['Post_Code'].astype(str).astype(int)

/tmp/ipykernel_441706/3008046163.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cook_locations_clean['Post_Code'] = cook_locations_clean['Post_Code'].astype(str).astype(int)


In [19]:
omec_locations = pd.read_csv('oemc_locations.csv')
omec_locations.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD


In [20]:
omec_locations.shape

(171266, 12)

In [21]:
omec_locations.dtypes

EventNumber          int64
EntryDate           object
EventType           object
TypeDescription     object
FinalDisposition    object
Location            object
CPDUnitList         object
CFDUnitList         object
numerical           object
directional         object
street_name         object
suffix              object
dtype: object

In [62]:
#check for alphabets in numerical column

In [22]:
omec_locations[omec_locations['numerical'].str.isalpha()]

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix
1250,2000801443,01/08/2020 03.15.54,THREAS,THREATENING SUICIDE,FILE,JESSE BROWN VA HOSPITAL,NaN,NaN,JESSE,BROWN,VA,HOSPITAL
33053,2017807536,06/26/2020 13.27.24,ASLTIP,ASSAULT IP,FILE,RIDGE & AUSTIN,NaN,NaN,RIDGE,&,NaN,AUSTIN
34323,2018417148,07/02/2020 22.13.10,ATSUCD,ATTEMPT SUICIDE,7I,TRUMP INTERNATIONAL HOTE,1834 1832,A42,TRUMP,INTERNATIONAL,NaN,HOTE
40557,2021610499,08/03/2020 17.14.11,NPS,NO POLICE SERVICE,FILE,JESSE BROWN VA HOSPITAL,NaN,NaN,JESSE,BROWN,VA,HOSPITAL
67246,2035401094,12/19/2020 01.30.42,THREAS,THREATENING SUICIDE,FILE,CLAYTON MOTEL/ SNOW AVENUE,LOVES PARK,IL,CLAYTON,MOTEL/,SNOW,AVENUE
77674,2104513267,02/14/2021 22.17.20,THREAS,THREATENING SUICIDE,FILE,NEW ENGLAND/ X TH,NaN,NaN,NEW,ENGLAND/,X,TH
98438,2114909460,05/29/2021 14.13.31,DISTME,MENTAL HEALTH DISTURBANCE,FILE,W 87TH ST,NaN,NaN,W,87TH,NaN,ST
108404,2119503444,07/14/2021 08.44.18,DISTME,MENTAL HEALTH DISTURBANCE,19P,ILLINOIS MCCLURG CT,1802S 1823 1834,NaN,ILLINOIS,MCCLURG,NaN,CT
109196,2119812628,07/17/2021 18.51.00,MISSPE,MISSING PERSON,NaN,TRAINING EXERCISE ONLY,NaN,NaN,TRAINING,EXERCISE,NaN,ONLY
113218,2121707009,08/05/2021 12.48.58,DEATHR,DEATH REMOVAL,NaN,SWEDISH COVENANT HOSPITAL,1701,NaN,SWEDISH,COVENANT,NaN,HOSPITAL


In [23]:
omec_locations[omec_locations['numerical'].str.isupper()]

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix
1250,2000801443,01/08/2020 03.15.54,THREAS,THREATENING SUICIDE,FILE,JESSE BROWN VA HOSPITAL,NaN,NaN,JESSE,BROWN,VA,HOSPITAL
33053,2017807536,06/26/2020 13.27.24,ASLTIP,ASSAULT IP,FILE,RIDGE & AUSTIN,NaN,NaN,RIDGE,&,NaN,AUSTIN
34323,2018417148,07/02/2020 22.13.10,ATSUCD,ATTEMPT SUICIDE,7I,TRUMP INTERNATIONAL HOTE,1834 1832,A42,TRUMP,INTERNATIONAL,NaN,HOTE
36638,2019615318,07/14/2020 21.29.09,SUSAUW,SUSPICIOUS AUTO WITH OCC,19PZ,N1198 N CICERO AV /48XX W DIVISION ST,1533,NaN,N1198,N,CICERO AV /48XX W DIVISION,ST
40557,2021610499,08/03/2020 17.14.11,NPS,NO POLICE SERVICE,FILE,JESSE BROWN VA HOSPITAL,NaN,NaN,JESSE,BROWN,VA,HOSPITAL
41322,2022004972,08/07/2020 10.44.10,CHILDP,CHILD PLAYING ON PHONE,FILE,A655 W 65TH ST,NaN,NaN,A655,W,65TH,ST
67246,2035401094,12/19/2020 01.30.42,THREAS,THREATENING SUICIDE,FILE,CLAYTON MOTEL/ SNOW AVENUE,LOVES PARK,IL,CLAYTON,MOTEL/,SNOW,AVENUE
77674,2104513267,02/14/2021 22.17.20,THREAS,THREATENING SUICIDE,FILE,NEW ENGLAND/ X TH,NaN,NaN,NEW,ENGLAND/,X,TH
97041,2114221111,05/22/2021 23.41.27,THREAS,THREATENING SUICIDE,FILE,AS5700-57XX W SCHOOL ST,NaN,NaN,AS5700-5700,W,SCHOOL,ST
98438,2114909460,05/29/2021 14.13.31,DISTME,MENTAL HEALTH DISTURBANCE,FILE,W 87TH ST,NaN,NaN,W,87TH,NaN,ST


In [24]:
omec_clean_locations = omec_locations[~omec_locations['numerical'].str.isupper()]

In [25]:
omec_clean_locations.shape

(171248, 12)

In [26]:
omec_clean_locations[~omec_clean_locations['numerical'].str.isnumeric()]

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix
21419,2011910398,04/28/2020 17.56.09,DIST,DISTURBANCE,5FZ,\2844 N BROADWAY,1935,NaN,\2844,N,NaN,BROADWAY


In [27]:
omec_clean_locations = omec_clean_locations[omec_clean_locations['EventNumber'] != 2011910398 ]

In [28]:
omec_clean_locations.shape

(171247, 12)

In [29]:
omec_clean_locations.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD


In [30]:
omec_clean_locations['numerical'] = omec_clean_locations['numerical'].astype(str).astype(int)

In [31]:
omec_clean_locations.dtypes

EventNumber          int64
EntryDate           object
EventType           object
TypeDescription     object
FinalDisposition    object
Location            object
CPDUnitList         object
CFDUnitList         object
numerical            int64
directional         object
street_name         object
suffix              object
dtype: object

In [32]:
#strip the any trailing white space
omec_clean_locations['directional'] = omec_clean_locations['directional'].str.strip()

In [33]:
omec_clean_locations.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD


In [34]:
def fullDirections(direction):
    if direction == 'W':
        return 'WEST'
    if direction == 'E':
        return 'EAST'
    if direction == 'S':
        return 'SOUTH'
    if direction == 'N':
        return 'NORTH'
    else:
        return direction
    

In [35]:
omec_clean_locations['St_PreDir']= omec_clean_locations['directional'].apply(fullDirections)
omec_clean_locations.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,St_PreDir
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST,SOUTH
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,SOUTH
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,WEST
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,.5
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,WEST


In [36]:
omec_clean_locations[omec_clean_locations['street_name'] == 'DR MARTIN LUTHER KING JR']['suffix'] = ''

/tmp/ipykernel_441706/1182088338.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  omec_clean_locations[omec_clean_locations['street_name'] == 'DR MARTIN LUTHER KING JR']['suffix'] = ''


In [37]:
omec_clean_locations.to_csv('cleaned_oemc_locations.csv')

In [38]:
#check out cook county address data and subset to columns needed for geocoding

In [39]:
address_points.head()

,OBJECTID,AddNum_Pre,Add_Number,AddNum_Suf,ADDRNOCOM,St_PreMod,St_PreDir,St_PreTyp,St_Name,St_PosTyp,...,Seat,Inc_Muni,Uninc_Comm,MSAGComm,DiscrpAgID,Site_NGUID,ESN,TWP_NAME,LSt_PreDir,LSt_Type
0,1,NaN,1763.0,NaN,1763,NaN,WEST,NaN,WELLINGTON,AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,1622.0,NaN,1622,NaN,WEST,NaN,CULLERTON,STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,917.0,NaN,917,NaN,WEST,NaN,ARMITAGE,AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,20018.0,NaN,20018,NaN,NaN,NaN,MOHAWK,TRAIL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,20024.0,NaN,20024,NaN,NaN,NaN,MOHAWK,TRAIL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
address_points.columns

Index(['OBJECTID', 'AddNum_Pre', 'Add_Number', 'AddNum_Suf', 'ADDRNOCOM',
       'St_PreMod', 'St_PreDir', 'St_PreTyp', 'St_Name', 'St_PosTyp',
       'St_PosDir', 'St_PosMod', 'STNAMECOM', 'LandmkName', 'PLACENAME',
       'Post_Comm', 'State', 'Post_Code', 'Post_Code4', 'ADDRDELIV',
       'CMPADDABRV', 'Long', 'Lat', 'USGRIDCORD', 'Elev', 'PIN', 'Effective',
       'Expire', 'EDITTIME', 'geocode_muni', 'Document_Name', 'created_user',
       'created_date', 'last_edited_user', 'last_edited_date', 'Source',
       'Source_Comment', 'Structure', 'OrthoYear', 'Placement', 'County',
       'Building', 'Floor', 'Unit', 'Room', 'Seat', 'Inc_Muni', 'Uninc_Comm',
       'MSAGComm', 'DiscrpAgID', 'Site_NGUID', 'ESN', 'TWP_NAME', 'LSt_PreDir',
       'LSt_Type'],
      dtype='object')

In [41]:
columns = ['CMPADDABRV', 'Lat', 'Long', 'PLACENAME', 'Post_Code', 'OBJECTID', 'Add_Number','St_PreDir', 'St_Name']

In [42]:
database = address_points[columns]
database 

,CMPADDABRV,Lat,Long,PLACENAME,Post_Code,OBJECTID,Add_Number,St_PreDir,St_Name
0,1763 W WELLINGTON AVE,41.935796,-87.673357,Chicago,60657,1,1763.0,WEST,WELLINGTON
1,1622 W CULLERTON ST,41.855258,-87.667139,Chicago,60608,2,1622.0,WEST,CULLERTON
2,917 W ARMITAGE AVE,41.917926,-87.651651,Chicago,60614,3,917.0,WEST,ARMITAGE
3,20018 MOHAWK TRL,41.525564,-87.695313,Olympia Fields,60461,4,20018.0,NaN,MOHAWK
4,20024 MOHAWK TRL,41.525317,-87.695269,Olympia Fields,60461,5,20024.0,NaN,MOHAWK
...,...,...,...,...,...,...,...,...,...
1347087,222 MERCHANDISE MART PLZ,NaN,NaN,CHICAGO,60654,1465915,222.0,NaN,MERCHANDISE MART
1347088,1410 S MUSEUM CAMPUS DR,NaN,NaN,CHICAGO,60605,1465916,1410.0,SOUTH,MUSEUM CAMPUS
1347089,6660 W 66TH ST,NaN,NaN,BEDFORD PARK,60638,1465917,NaN,WEST,66TH
1347090,9590 W 55TH ST,NaN,NaN,COUNTRYSIDE,60525,1465918,9590.0,WEST,55TH


In [43]:
database = database[database['PLACENAME'] =='Chicago']

In [44]:
database.shape

(582251, 9)

In [45]:
#split array into chunks in order to run the process on my laptop

In [46]:
chunkDf = np.array_split(omec_clean_locations,30)
chunkDf

[      EventNumber            EntryDate EventType            TypeDescription  \
 0      2000100015  01/01/2020 00.00.52    DISTME  MENTAL HEALTH DISTURBANCE   
 1      2000100451  01/01/2020 00.21.18      DIST                DISTURBANCE   
 2      2000100656  01/01/2020 00.33.55    DOMBAT           DOMESTIC BATTERY   
 3      2000100660  01/01/2020 00.34.00    DISTME  MENTAL HEALTH DISTURBANCE   
 4      2000100902  01/01/2020 00.47.02    CHECWB           CHECK WELL BEING   
 ...           ...                  ...       ...                        ...   
 5705   2003312092  02/02/2020 17.59.38    DISTME  MENTAL HEALTH DISTURBANCE   
 5706   2003312108  02/02/2020 18.00.43    ASLTIP                 ASSAULT IP   
 5707   2003312147  02/02/2020 18.03.29      DIST                DISTURBANCE   
 5708   2003312242  02/02/2020 18.11.31    CHECWB           CHECK WELL BEING   
 5709   2003312295  02/02/2020 18.17.03    DISTME  MENTAL HEALTH DISTURBANCE   
 
      FinalDisposition                

In [47]:
#future use case this will not work for exact address
#do not search for address bewtween 100 off
def geoCodeChunk(number,chunkDfName,database):
    list_zipcodes = []
    for index, row in chunkDfName[number].iterrows():
        complete_conditional = database[(database['St_Name'] == row['street_name']) 
                                    & (database['St_PreDir'] == row['St_PreDir'])
                                    & (database['Add_Number'] < row.numerical + 100)
                                    & (database['Add_Number'] >= row.numerical -100)
                                   ]
        list_zipcodes.append(list(set(complete_conditional['Post_Code'])))
    #print(set(complete_conditional['Post_Code']))
    print('Done')
    print("--- %s minutes ---" % ((time.time() - start_time)/60))
    return list_zipcodes
    

In [48]:
def geoCodeToCSV(number,chunkDfName,chunkZipCodeList):
    chunkDfName[number]['zipcodes']= chunkZipCodeList
    chunkDfName[number].to_csv(f'geocoded_batches/batch{number}.csv')
    

In [49]:
#creates 30 empty list 
#name convention list_zipcodes0, list_zipcodes1,..etc
for k in range(30):
    exec(f'list_zipcodes{k} = []')


## Testing code

In [54]:
testChunk = chunkDf[0].iloc[1:10]
testChunk.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,St_PreDir
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,SOUTH
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,WEST
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,.5
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,WEST
5,2000100939,01/01/2020 00.49.17,CHECWB,CHECK WELL BEING,19B,50XX N SHERIDAN RD,2041R 2024R,NaN,5000,N,SHERIDAN,RD,NORTH


In [ ]:
geoCodeChunk(number,chunkDfName,database)

In [71]:
test_zipcodes = []
for index, row in testChunk.iterrows():
    complete_conditional = database[(database['St_Name'] == row['street_name']) 
                                & (database['St_PreDir'] == row['St_PreDir'])
                                & (database['Add_Number'] < row.numerical + 100)
                                & (database['Add_Number'] >= row.numerical -100)
                                ]
    test_zipcodes.append(list(set(complete_conditional['Post_Code'])))
    #print(set(complete_conditional['Post_Code']))
print('Done')
#print("--- %s minutes ---" % ((time.time() - start_time)/60))


Done


In [57]:
testChunk['zipcodes'] = test_zipcodes
testChunk.to_csv(f'geocoded_batches/batch_test.csv')


/tmp/ipykernel_441706/893343377.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testChunk['zipcodes'] = test_zipcodes


In [59]:
testChunk.dtypes

EventNumber          int64
EntryDate           object
EventType           object
TypeDescription     object
FinalDisposition    object
Location            object
CPDUnitList         object
CFDUnitList         object
numerical            int64
directional         object
street_name         object
suffix              object
St_PreDir           object
zipcodes            object
dtype: object

In [60]:
testChunk

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,St_PreDir,zipcodes
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,SOUTH,[60655]
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,WEST,"[60641, 60618]"
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,.5,[]
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,WEST,[60634]
5,2000100939,01/01/2020 00.49.17,CHECWB,CHECK WELL BEING,19B,50XX N SHERIDAN RD,2041R 2024R,NaN,5000,N,SHERIDAN,RD,NORTH,[60640]
6,2000100941,01/01/2020 00.49.19,BATIP,BATTERY IP,1EZ,66XX W BELDEN AV,2512R 2510R,NaN,6600,W,BELDEN,AV,WEST,[60707]
7,2000100959,01/01/2020 00.50.02,THREAS,THREATENING SUICIDE,16I,5XX N AUSTIN BL,1511R 1512R,NaN,500,N,AUSTIN,BL,NORTH,[60644]
8,2000100999,01/01/2020 00.52.12,DISTME,MENTAL HEALTH DISTURBANCE,5079Z,50XX S MICHIGAN AV,224R,NaN,5000,S,MICHIGAN,AV,SOUTH,[60615]
9,2000101032,01/01/2020 00.53.27,DD,DOMESTIC DISTURBANCE,1IZ,10XX N DEARBORN ST,1824R 1823R,NaN,1000,N,DEARBORN,ST,NORTH,[60610]


In [69]:
testChunk[['zipcodes']].iloc[0][0][0]

60655

In [70]:
testChunk[['zipcodes']].iloc[1][0][0]

60641

In [51]:
for i in range(1):
    exec(f'list_zipcodes{i} = geoCodeChunk(i,chunkDf,database)')
    exec(f'geoCodeToCSV(i,chunkDf,list_zipcodes{i})')

KeyboardInterrupt: 

## Real code

In [53]:
for i in range(6):
    exec(f'list_zipcodes{i} = geoCodeChunk(i,chunkDf,database)')
    exec(f'geoCodeToCSV(i,chunkDf,list_zipcodes{i})')
    

Done
Done
Done
Done
Done
Done


In [57]:
for i in range(6,12):
    exec(f'list_zipcodes{i} = geoCodeChunk(i,chunkDf,database)')
    exec(f'geoCodeToCSV(i,chunkDf,list_zipcodes{i})')

KeyboardInterrupt: 

In [80]:
for i in range(12,18):
    exec(f'list_zipcodes{i} = geoCodeChunk(i,chunkDf,database)')
    exec(f'geoCodeToCSV(i,chunkDf,list_zipcodes{i})')

Done
Done
Done
Done
Done
Done


In [81]:
for i in range(18,24):
    exec(f'list_zipcodes{i} = geoCodeChunk(i,chunkDf,database)')
    exec(f'geoCodeToCSV(i,chunkDf,list_zipcodes{i})')

Done
Done
Done
Done
Done
Done


In [82]:
for i in range(24,30):
    exec(f'list_zipcodes{i} = geoCodeChunk(i,chunkDf,database)')
    exec(f'geoCodeToCSV(i,chunkDf,list_zipcodes{i})')

Done
Done
Done
Done
Done
Done


## Code Chunks

In [ ]:
#chunkDf[0]

In [55]:
import time

In [64]:
start_time = time.time()
list_zipcodes2 = []
for index, row in chunkDf[2].iterrows():
    complete_conditional = database[(database['St_Name'] == row['street_name']) 
                                    & (database['St_PreDir'] == row['St_PreDir'])
                                    & (database['Add_Number'] < row.numerical + 100)
                                    & (database['Add_Number'] > row.numerical -100)
                                   ]
    list_zipcodes2.append(set(complete_conditional['Post_Code']))
    #print(set(complete_conditional['Post_Code']))
print('Done')
print("--- %s seconds ---" % (time.time() - start_time))
print("--- %s minutes ---" % ((time.time() - start_time)/60))


Done
--- 627.6527528762817 seconds ---
--- 10.460880482196808 minutes ---


In [57]:
chunkDf[2]['zipcodes']= list_zipcodes2

chunkDf[2].to_csv('batch2.csv')